In [1]:
import sys

import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision.transforms as T

import glob
import os

from skimage.io import imread, imsave, imshow

In [6]:

dataDir = "/users/btorres/databases/ecommerce/train/"
pidinet = "/home/vision/smb-datasets/ecommerce/ecommerce_pidinet/"
saveDir = "/users/btorres/databases/ecommerce/pidinet_color_quanta16_layer_10hues/"


In [7]:
def quant(imgen, cuant):
    cuant = 256 /cuant
    return ((torch.floor(imgen / cuant) * cuant) + (cuant/2))

def blend(pidimg, imagen):

    alpha = pidimg / 255

    black = torch.zeros(pidimg.shape).to('cuda') + 20

    result = imagen

    result[:,:,0] =  alpha * imagen[:,:,0] + (1 - alpha) * black
    result[:,:,1] =  alpha * imagen[:,:,1] + (1 - alpha) * black
    result[:,:,2] =  alpha * imagen[:,:,2] + (1 - alpha) * black

    return result


In [ ]:
o = 0
for class_path in glob.glob(os.path.join(dataDir, '**')):

    class_name = os.path.basename(class_path) + "/"

    newpath = saveDir + class_name
    if not os.path.exists(newpath[:-1]):
        os.makedirs(newpath)

    for sketch_path in glob.iglob(os.path.join(class_path, '**')):
        image_name = os.path.basename(sketch_path)
        save_path = saveDir + class_name + "/" + image_name.split(".")[-2] + ".jpg"

        l = image_name.split("_")
        pidinet_name = "_".join(l[0:-1]) if len(l) > 1 else l[0]


        pidinet_path = pidinet + class_name + "/" + pidinet_name.split(".")[0] + ".jpg"

        img = torch.from_numpy(imread(sketch_path)).to('cuda')

        try:
            pidinet_img = torch.from_numpy(imread(pidinet_path)).to('cuda')

        except Exception as e:
            print(image_name)
            print(pidinet_path)
            print(save_path)

            print(e)

            break



        q = img
        q = quant(q, 16).type(torch.int)
        q = blend(pidinet_img, q)

        img_warp = q

        #img_warp = torch.cat((q, torch.unsqueeze(pidinet_img, dim = 2)), dim=2)
        #img_warp = img_warp / img_warp.max() #normalizes img_grey in range 0 - 255
        img_warp = img_warp.type(torch.uint8)
        resultado= img_warp.to('cpu').numpy()


        imsave(save_path, resultado)

        if (o % 100 == 0):

            sys.stdout.write('\r {} imágenes procesadas, clase: {}'.format( o, class_name[0:-1]))

        o += 1


 300 imágenes procesadas, clase: tazas